In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from os.path import basename
# !pip install boltons
from boltons.fileutils import iter_find_files

df_train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv', usecols=['cp_type'])
df_target_s = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

y = df_target_s.drop('sig_id', axis=1)
y = y[df_train['cp_type'] == 'trt_cp'].reset_index(drop = True)

print('train target scored size', y.shape)
def log_loss_metric(y_true, y_pred, columns=None):
    metrics = []
    cols = y_true.columns if columns is None else columns
    ypred_vals = y_pred[cols].values
    yvals = y_true.values
    
    for i, _target in enumerate(cols):
        metrics.append(
            log_loss(
                yvals[:, i],
                ypred_vals[:, i],
                labels = [0, 1],
                eps=1e-15
            )
        )
    return np.mean(metrics)

train target scored size (21948, 206)


In [2]:
bundles = []
for bundle_file in iter_find_files('bundles', '*.pickle'):
    bundle = pickle.load(open(bundle_file, 'rb'))
    if bundle['logloss_valid'] > 0.0172:
        continue
    bundles.append({
        'filepath': basename(bundle_file),
        'predictions': bundle['predictions'],
        'preprocess_params': bundle['preprocess_params'],
        'deep_params': bundle['deep_params'],
        'blend_params': bundle['blend_params'],
        'seed': bundle['seed']
    })
    del bundle

print(len(bundles))

10


In [3]:
from itertools import combinations
from multiprocessing import Pool
from boltons.iterutils import flatten

scores = []
best_score = 1.
for m in bundles:
    for mpred_type, mpred in m['predictions'].items():
        loss = log_loss_metric(y, mpred)
        binfo = {'bundles': [(m["filepath"], mpred_type)], 'loss': loss}
        if loss < best_score:
            print(f'best score: {binfo}')
            best_score = loss
        scores.append(binfo)

def blend_mean(vals):
    return np.mean(vals, axis=0)

def blend_max(vals):
    return np.max(vals, axis=0)

def blend_min(vals):
    return np.min(vals, axis=0)

def get_scores2(arg):
    m1_ind, m2_ind = arg
    scores = []
    m1 = bundles[m1_ind]
    m2 = bundles[m2_ind]
    best_score = 1.
    for m1pred_type, m1pred in m1['predictions'].items():
        m1vals = m1pred.values
        for m2pred_type, m2pred in m2['predictions'].items():
            m2vals = m2pred.values
            for agg in [blend_mean, blend_max, blend_min]:
                newpred = agg([m1vals, m2vals])
                comb = pd.DataFrame(newpred, columns=y.columns)
                loss = log_loss_metric(y, comb)
                res = {
                    'bundles': [(m1["filepath"], m1pred_type), (m2["filepath"], m2pred_type)],
                    'loss': log_loss_metric(y, comb),
                    'agg': str(agg)
                }
                if loss < best_score:
                    print('')
                    print(f'best score: {res}')
                    print('')
                    best_score = loss
                print(str(res))
                scores.append(res)
    return scores

def get_scores3(arg):
    m1_ind, m2_ind, m3_ind = arg
    scores = []
    m1 = bundles[m1_ind]
    m2 = bundles[m2_ind]
    m3 = bundles[m3_ind]
    best_score = 1.
    for m1pred_type, m1pred in m1['predictions'].items():
        m1vals = m1pred.values
        for m2pred_type, m2pred in m2['predictions'].items():
            m2vals = m2pred.values
            for m3pred_type, m3pred in m3['predictions'].items():
                m3vals = m3pred.values
                for agg in [blend_mean]:
                    newpred = agg([m1vals, m2vals, m3vals])
                    comb = pd.DataFrame(newpred, columns=y.columns)
                    loss = log_loss_metric(y, comb)
                    res = {
                        'bundles': [
                            (m1["filepath"], m1pred_type), 
                            (m2["filepath"], m2pred_type),
                            (m3["filepath"], m3pred_type)
                        ],
                        'loss': log_loss_metric(y, comb),
                        'agg': str(agg)
                    }
                    if loss < best_score:
                        print('')
                        print(f'best score: {res}')
                        print('')
                        best_score = loss
                    print(str(res))
                    scores.append(res)
    return scores


best score: {'bundles': [('1606664066.pickle', 'initial')], 'loss': 0.017183086456201355}
best score: {'bundles': [('1606581561.pickle', 'initial')], 'loss': 0.016566159961711777}
best score: {'bundles': [('1606572570.pickle', 'initial')], 'loss': 0.016288387098946904}
best score: {'bundles': [('1606584530.pickle', 'initial')], 'loss': 0.016273160241381653}


In [4]:
pool = Pool(12)
scores2 = pool.map(get_scores2, list(combinations(range(len(bundles)), 2)))
scores += [s2 for s1 in scores2 for s2 in s1]


best score: {'bundles': [('1606664066.pickle', 'initial'), ('1606682121.pickle', 'initial')], 'loss': 0.016628518893604196, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

{'bundles': [('1606664066.pickle', 'initial'), ('1606682121.pickle', 'initial')], 'loss': 0.016628518893604196, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

best score: {'bundles': [('1606664066.pickle', 'initial'), ('1606581561.pickle', 'initial')], 'loss': 0.01636218675639331, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

{'bundles': [('1606664066.pickle', 'initial'), ('1606581561.pickle', 'initial')], 'loss': 0.01636218675639331, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

best score: {'bundles': [('1606664066.pickle', 'initial'), ('1606702305.pickle', 'initial')], 'loss': 0.016206610571667272, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

{'bundles': [('1606664066.pickle', 'initial'), ('1606702305.pickle', 'initial')], 'loss': 0.016206610571667272, 'agg': '<function blend_mean at 0x7fbf

{'bundles': [('1606581561.pickle', 'initial'), ('1606696454.pickle', 'after_zero')], 'loss': 0.016190426109917742, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606581561.pickle', 'final')], 'loss': 0.01636218675639331, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606692786.pickle', 'after_zero')], 'loss': 0.01653222776244663, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606696454.pickle', 'after_zero')], 'loss': 0.016498672521463458, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606637843.pickle', 'after_zero')], 'loss': 0.01690556307696026, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606584530.pickle', 'final')], 'loss': 0.016390500443601238, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('16065815

{'bundles': [('1606581561.pickle', 'final'), ('1606572570.pickle', 'initial')], 'loss': 0.016399918492028077, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606692786.pickle', 'after_error')], 'loss': 0.0168578407969219, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606581561.pickle', 'initial')], 'loss': 0.01660690486274769, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606637843.pickle', 'final')], 'loss': 0.01703453464640107, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606580084.pickle', 'final')], 'loss': 0.017145663772522093, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606696454.pickle', 'after_error')], 'loss': 0.016827780268791066, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606581561.pickle'

{'bundles': [('1606581561.pickle', 'final'), ('1606572570.pickle', 'final')], 'loss': 0.016454628568630603, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606696454.pickle', 'final')], 'loss': 0.016990456792642605, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606637843.pickle', 'initial')], 'loss': 0.017035788027889718, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606580084.pickle', 'initial')], 'loss': 0.017167692158022194, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606572570.pickle', 'final')], 'loss': 0.01689143470487927, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606702305.pickle', 'final')], 'loss': 0.016994968287569967, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606581561.pickle', '

{'bundles': [('1606664066.pickle', 'after_error'), ('1606584530.pickle', 'final')], 'loss': 0.016390500443601238, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606696454.pickle', 'after_zero')], 'loss': 0.016498672521463458, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606581561.pickle', 'final'), ('1606682121.pickle', 'after_zero')], 'loss': 0.01619136333968235, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606702305.pickle', 'after_zero')], 'loss': 0.016206610571667272, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606682121.pickle', 'after_zero')], 'loss': 0.016828836402375574, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606580084.pickle', 'final')], 'loss': 0.01712485292242882, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles

...

{'bundles': [('1606696454.pickle', 'final'), ('1606702305.pickle', 'final')], 'loss': 0.016548912782860328, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606692786.pickle', 'initial'), ('1606702305.pickle', 'after_error')], 'loss': 0.016161041209946943, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606580084.pickle', 'final'), ('1606692786.pickle', 'initial')], 'loss': 0.01651929204083445, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606692786.pickle', 'after_error'), ('1606584530.pickle', 'initial')], 'loss': 0.016112878998508384, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606580084.pickle', 'after_error'), ('1606702305.pickle', 'initial')], 'loss': 0.016460094361074644, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606682121.pickle', 'final'), ('1606692786.pickle', 'after_error')], 'loss': 0.0167499152537408, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606584530.

{'bundles': [('1606584530.pickle', 'initial'), ('1606702305.pickle', 'after_zero')], 'loss': 0.01641591368367597, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606692786.pickle', 'after_zero'), ('1606637843.pickle', 'after_zero')], 'loss': 0.016803867020163846, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606584530.pickle', 'initial'), ('1606637843.pickle', 'after_zero')], 'loss': 0.01657113213791029, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606682121.pickle', 'final'), ('1606692786.pickle', 'final')], 'loss': 0.01657761252966631, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606637843.pickle', 'initial'), ('1606702305.pickle', 'after_zero')], 'loss': 0.016469481596578805, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606580084.pickle', 'after_error'), ('1606702305.pickle', 'after_zero')], 'loss': 0.01694723761718072, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('16065

{'bundles': [('1606692786.pickle', 'after_error'), ('1606637843.pickle', 'initial')], 'loss': 0.016494811193320295, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606580084.pickle', 'final'), ('1606692786.pickle', 'final')], 'loss': 0.01651929204083445, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606692786.pickle', 'final'), ('1606584530.pickle', 'final')], 'loss': 0.016112878998508384, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'final'), ('1606702305.pickle', 'final')], 'loss': 0.01612085585641618, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606580084.pickle', 'after_error'), ('1606702305.pickle', 'final')], 'loss': 0.016460094361074644, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606584530.pickle', 'initial'), ('1606702305.pickle', 'final')], 'loss': 0.01613430906161811, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606584530.pickle', 'final

{'bundles': [('1606692786.pickle', 'after_error'), ('1606637843.pickle', 'final')], 'loss': 0.016803867020163846, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606637843.pickle', 'after_zero'), ('1606702305.pickle', 'after_zero')], 'loss': 0.016469481596578805, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606692786.pickle', 'after_zero'), ('1606702305.pickle', 'final')], 'loss': 0.016385078317525305, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606584530.pickle', 'final'), ('1606637843.pickle', 'final')], 'loss': 0.01657113213791029, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606584530.pickle', 'final'), ('1606702305.pickle', 'after_zero')], 'loss': 0.01641591368367597, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606580084.pickle', 'final'), ('1606702305.pickle', 'after_zero')], 'loss': 0.01694723761718072, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606692786.pick

{'bundles': [('1606637843.pickle', 'final'), ('1606702305.pickle', 'after_zero')], 'loss': 0.016469481596578805, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606692786.pickle', 'after_error'), ('1606702305.pickle', 'final')], 'loss': 0.016385078317525305, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606637843.pickle', 'final'), ('1606702305.pickle', 'after_zero')], 'loss': 0.016694817125423058, 'agg': '<function blend_min at 0x7fbfe31ef4c0>'}
{'bundles': [('1606637843.pickle', 'final'), ('1606702305.pickle', 'after_error')], 'loss': 0.01614497296607532, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606692786.pickle', 'final'), ('1606702305.pickle', 'initial')], 'loss': 0.016161041209946943, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606637843.pickle', 'final'), ('1606702305.pickle', 'after_error')], 'loss': 0.016469481596578805, 'agg': '<function blend_max at 0x7fbfe31ef5e0>'}
{'bundles': [('1606692786.p

In [5]:
pool = Pool(12)
scores3 = pool.map(get_scores3, list(combinations(range(len(bundles)), 3)))
scores += [s2 for s1 in scores3 for s2 in s1]


best score: {'bundles': [('1606664066.pickle', 'initial'), ('1606682121.pickle', 'initial'), ('1606637843.pickle', 'initial')], 'loss': 0.01661013969605938, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

{'bundles': [('1606664066.pickle', 'initial'), ('1606682121.pickle', 'initial'), ('1606637843.pickle', 'initial')], 'loss': 0.01661013969605938, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

best score: {'bundles': [('1606664066.pickle', 'initial'), ('1606581561.pickle', 'initial'), ('1606572570.pickle', 'initial')], 'loss': 0.01610476932491225, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

{'bundles': [('1606664066.pickle', 'initial'), ('1606581561.pickle', 'initial'), ('1606572570.pickle', 'initial')], 'loss': 0.01610476932491225, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

best score: {'bundles': [('1606664066.pickle', 'initial'), ('1606581561.pickle', 'initial'), ('1606580084.pickle', 'initial')], 'loss': 0.01645395715241867, 'agg': '<function blend_mean at 0x

{'bundles': [('1606664066.pickle', 'initial'), ('1606580084.pickle', 'after_error'), ('1606584530.pickle', 'initial')], 'loss': 0.016522750096541607, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606682121.pickle', 'initial'), ('1606637843.pickle', 'final')], 'loss': 0.01661013969605938, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606696454.pickle', 'initial'), ('1606682121.pickle', 'after_error')], 'loss': 0.016436289420558768, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606682121.pickle', 'initial'), ('1606580084.pickle', 'final')], 'loss': 0.016677154239821478, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606692786.pickle', 'after_zero'), ('1606584530.pickle', 'initial')], 'loss': 0.01621086373536064, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}{'bundles': [('1

{'bundles': [('1606664066.pickle', 'initial'), ('1606696454.pickle', 'after_zero'), ('1606682121.pickle', 'after_zero')], 'loss': 0.016436289420558768, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606580084.pickle', 'final'), ('1606584530.pickle', 'final')], 'loss': 0.016522750096541607, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606584530.pickle', 'final'), ('1606637843.pickle', 'final')], 'loss': 0.016448507522333906, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606581561.pickle', 'initial'), ('1606580084.pickle', 'initial')], 'loss': 0.01645395715241867, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606682121.pickle', 'after_error'), ('1606637843.pickle', 'initial')], 'loss': 0.01661013969605938, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('16

{'bundles': [('1606664066.pickle', 'initial'), ('1606682121.pickle', 'final'), ('1606637843.pickle', 'initial')], 'loss': 0.01661013969605938, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'after_error'), ('1606572570.pickle', 'initial'), ('1606584530.pickle', 'final')], 'loss': 0.016077722840535175, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'after_error'), ('1606581561.pickle', 'initial'), ('1606572570.pickle', 'final')], 'loss': 0.01610476932491225, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'after_zero'), ('1606581561.pickle', 'final'), ('1606637843.pickle', 'initial')], 'loss': 0.016385785369510786, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606696454.pickle', 'after_error'), ('1606682121.pickle', 'after_zero')], 'loss': 0.016436289420558768, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': 

{'bundles': [('1606664066.pickle', 'after_zero'), ('1606692786.pickle', 'after_error'), ('1606584530.pickle', 'initial')], 'loss': 0.01621086373536064, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'final'), ('1606581561.pickle', 'initial'), ('1606572570.pickle', 'initial')], 'loss': 0.01610476932491225, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'final'), ('1606572570.pickle', 'initial'), ('1606584530.pickle', 'initial')], 'loss': 0.016077722840535175, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'initial'), ('1606696454.pickle', 'final'), ('1606682121.pickle', 'initial')], 'loss': 0.016436289420558768, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606664066.pickle', 'after_error'), ('1606580084.pickle', 'initial'), ('1606584530.pickle', 'initial')], 'loss': 0.016522750096541607, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('16

...

{'bundles': [('1606696454.pickle', 'final'), ('1606682121.pickle', 'initial'), ('1606702305.pickle', 'final')], 'loss': 0.01614151138579464, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'final'), ('1606580084.pickle', 'final'), ('1606692786.pickle', 'after_zero')], 'loss': 0.016453923142397804, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

best score: {'bundles': [('1606580084.pickle', 'initial'), ('1606584530.pickle', 'initial'), ('1606637843.pickle', 'initial')], 'loss': 0.01644180809677985, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}

{'bundles': [('1606580084.pickle', 'initial'), ('1606584530.pickle', 'initial'), ('1606637843.pickle', 'initial')], 'loss': 0.01644180809677985, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606696454.pickle', 'final'), ('1606682121.pickle', 'final'), ('1606692786.pickle', 'final')], 'loss': 0.016447334846412996, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('16066

{'bundles': [('1606580084.pickle', 'after_error'), ('1606692786.pickle', 'initial'), ('1606584530.pickle', 'final')], 'loss': 0.016205806473475994, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'after_error'), ('1606692786.pickle', 'after_error'), ('1606702305.pickle', 'after_error')], 'loss': 0.016156595013029445, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'initial'), ('1606580084.pickle', 'after_error'), ('1606584530.pickle', 'final')], 'loss': 0.01625372518853507, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606584530.pickle', 'initial'), ('1606637843.pickle', 'initial'), ('1606702305.pickle', 'after_zero')], 'loss': 0.01596639586037453, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606580084.pickle', 'initial'), ('1606584530.pickle', 'final'), ('1606637843.pickle', 'final')], 'loss': 0.01644180809677985, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundle

{'bundles': [('1606682121.pickle', 'after_error'), ('1606692786.pickle', 'final'), ('1606702305.pickle', 'final')], 'loss': 0.016156595013029445, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'after_error'), ('1606692786.pickle', 'after_zero'), ('1606584530.pickle', 'final')], 'loss': 0.016148327665818932, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606584530.pickle', 'initial'), ('1606637843.pickle', 'after_zero'), ('1606702305.pickle', 'final')], 'loss': 0.01596639586037453, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'final'), ('1606692786.pickle', 'initial'), ('1606702305.pickle', 'initial')], 'loss': 0.016156595013029445, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606580084.pickle', 'after_error'), ('1606692786.pickle', 'final'), ('1606584530.pickle', 'final')], 'loss': 0.016205806473475994, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('16

{'bundles': [('1606580084.pickle', 'final'), ('1606584530.pickle', 'final'), ('1606637843.pickle', 'final')], 'loss': 0.01644180809677985, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606580084.pickle', 'final'), ('1606692786.pickle', 'after_error'), ('1606584530.pickle', 'final')], 'loss': 0.016205806473475994, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'final'), ('1606692786.pickle', 'initial'), ('1606584530.pickle', 'final')], 'loss': 0.016148327665818932, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'final'), ('1606692786.pickle', 'after_zero'), ('1606584530.pickle', 'initial')], 'loss': 0.016148327665818932, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'after_error'), ('1606580084.pickle', 'final'), ('1606584530.pickle', 'initial')], 'loss': 0.01625372518853507, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606580084.pic

{'bundles': [('1606682121.pickle', 'initial'), ('1606584530.pickle', 'final'), ('1606637843.pickle', 'initial')], 'loss': 0.016237427678250805, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'initial'), ('1606584530.pickle', 'final'), ('1606637843.pickle', 'after_zero')], 'loss': 0.016237427678250805, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'initial'), ('1606584530.pickle', 'final'), ('1606637843.pickle', 'final')], 'loss': 0.016237427678250805, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'after_zero'), ('1606584530.pickle', 'initial'), ('1606637843.pickle', 'initial')], 'loss': 0.016237427678250805, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1606682121.pickle', 'after_zero'), ('1606584530.pickle', 'initial'), ('1606637843.pickle', 'after_zero')], 'loss': 0.016237427678250805, 'agg': '<function blend_mean at 0x7fbfe3209e50>'}
{'bundles': [('1

In [30]:
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(scores).sort_values('loss')
df

,bundles,loss,agg
3818,"[(1606572570.pickle, initial), (1606584530.pickle, initial), (1606702305.pickle, initial)]",0.015799,<function blend_mean at 0x7fbfe3209e50>
3821,"[(1606572570.pickle, initial), (1606584530.pickle, initial), (1606702305.pickle, final)]",0.015799,<function blend_mean at 0x7fbfe3209e50>
3822,"[(1606572570.pickle, initial), (1606584530.pickle, final), (1606702305.pickle, initial)]",0.015799,<function blend_mean at 0x7fbfe3209e50>
3823,"[(1606572570.pickle, initial), (1606584530.pickle, final), (1606702305.pickle, after_zero)]",0.015799,<function blend_mean at 0x7fbfe3209e50>
3824,"[(1606572570.pickle, initial), (1606584530.pickle, final), (1606702305.pickle, after_error)]",0.015799,<function blend_mean at 0x7fbfe3209e50>
...,...,...,...
187,"[(1606664066.pickle, after_zero), (1606580084.pickle, initial)]",0.017168,<function blend_min at 0x7fbfe31ef4c0>
3,"[(1606664066.pickle, final)]",0.017183,NaN
2,"[(1606664066.pickle, after_error)]",0.017183,NaN
1,"[(1606664066.pickle, after_zero)]",0.017183,NaN


In [31]:
final_conf = []
for r in df.bundles.values[0]:
    print(r)
    bundle = [b for b in bundles if b['filepath'] == r[0]][0]
    args = (bundle['preprocess_params'], bundle['deep_params'], bundle['blend_params'], bundle['seed'], 1)
    final_conf.append(args)
print(final_conf)

('1606572570.pickle', 'initial')
('1606584530.pickle', 'initial')
('1606702305.pickle', 'initial')
[({'fe_cluster': {'enabled': False, 'n_clusters_c': 6, 'n_clusters_g': 44}, 'fe_stats': {'enabled': False}, 'pca': {'enabled': True, 'n_comp_cells': 50, 'n_comp_genes': 600}, 'scaler': 'quantile', 'shuffle_cols': True, 'variance_reduction': {'enabled': True, 'threshold': 0.8}}, {'batch_norm': True, 'weight_norm': True, 'activation': ['elu', 'elu', 'elu', 'elu'], 'dropout': [0.3, 0.3, 0.4, 0.3], 'hid_layer': [512, 1024, 512, 2048], 'init_non_scored_weights': False, 'label_smoothing': 0.0015, 'learning_rate': 0.001, 'epochs': 35}, {'use_error_class': False, 'zero_threshold': 0}, 8, 1), ({'fe_cluster': {'enabled': True, 'n_clusters_c': 4, 'n_clusters_g': 22}, 'fe_stats': {'enabled': True}, 'pca': {'enabled': True, 'n_comp_cells': 50, 'n_comp_genes': 600, 'n_clusters': 5}, 'scaler': 'quantile', 'shuffle_cols': False, 'variance_reduction': {'enabled': True, 'threshold': 0.85}}, {'batch_norm': 

In [ ]:
[
    ({'fe_cluster': {'enabled': False, 'n_clusters_c': 6, 'n_clusters_g': 44}, 'fe_stats': {'enabled': False}, 'pca': {'enabled': True, 'n_comp_cells': 50, 'n_comp_genes': 600}, 'scaler': 'quantile', 'shuffle_cols': True, 'variance_reduction': {'enabled': True, 'threshold': 0.8}}, {'batch_norm': True, 'weight_norm': True, 'activation': ['elu', 'elu', 'elu', 'elu'], 'dropout': [0.3, 0.3, 0.4, 0.3], 'hid_layer': [512, 1024, 512, 2048], 'init_non_scored_weights': False, 'label_smoothing': 0.0015, 'learning_rate': 0.001, 'epochs': 35}, {'use_error_class': False, 'zero_threshold': 0}, 8, 1), 
    ({'fe_cluster': {'enabled': True, 'n_clusters_c': 4, 'n_clusters_g': 22}, 'fe_stats': {'enabled': True}, 'pca': {'enabled': True, 'n_comp_cells': 50, 'n_comp_genes': 600, 'n_clusters': 5}, 'scaler': 'quantile', 'shuffle_cols': False, 'variance_reduction': {'enabled': True, 'threshold': 0.85}}, {'batch_norm': True, 'weight_norm': 'output', 'activation': ['leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu'], 'dropout': [0.5, 0.35, 0.3, 0.25], 'hid_layer': [1500, 1250, 1000, 750], 'init_non_scored_weights': True, 'label_smoothing': 0, 'learning_rate': 0.001, 'epochs': 25}, {'use_error_class': False, 'zero_threshold': 0}, 42, 1), 
    ({'fe_cluster': {'enabled': False, 'n_clusters_c': 6, 'n_clusters_g': 44}, 'fe_stats': {'enabled': False}, 'pca': {'enabled': True, 'n_comp_cells': 60, 'n_comp_genes': 463}, 'scaler': 'quantile', 'shuffle_cols': False, 'variance_reduction': {'enabled': True, 'threshold': 0.9}}, {'batch_norm': True, 'weight_norm': True, 'activation': ['leaky_relu', 'leaky_relu'], 'dropout': [0.25, 0.25], 'hid_layer': [1500, 1500], 'init_non_scored_weights': False, 'label_smoothing': 0, 'learning_rate': 0.001, 'epochs': 35}, {'use_error_class': False, 'zero_threshold': 0}, 0, 1)
]
